# Наискорейший спуск на сфере
## Постановка задачи
$$\underset{\| x \|^2 - 1 = 0}{min} f(x) = x^T A x$$


In [1]:
import numpy as np

## В следующей ячейке в первой строке задается матрица квадратичной формы.
### Далее заранее обсчитываются квадрат и куб этой матрицы, задается начальное приближение, потом сразу же проецируется на сферу.

In [82]:
A = np.array([[1., 1., -1.], [1., 100., 0], [-1.,0,1.]])
A_sq = A.dot(A)
A_th = A.dot(A_sq)
x0 = np.array([1., 0, 1.])
x0 = x0 / np.linalg.norm(x0)

In [83]:
print A
print A_sq
print A_th

[[   1.    1.   -1.]
 [   1.  100.    0.]
 [  -1.    0.    1.]]
[[  3.00000000e+00   1.01000000e+02  -2.00000000e+00]
 [  1.01000000e+02   1.00010000e+04  -1.00000000e+00]
 [ -2.00000000e+00  -1.00000000e+00   2.00000000e+00]]
[[  1.06000000e+02   1.01030000e+04  -5.00000000e+00]
 [  1.01030000e+04   1.00020100e+06  -1.02000000e+02]
 [ -5.00000000e+00  -1.02000000e+02   4.00000000e+00]]


## В следующей ячейке задается функция, которая возвращает значение второй производной целевой функции при выборе шага (см pdf)
$$F'' = \frac{\gamma^3 (-2a_2\widehat{a_2} )+ \gamma^2 (-3a_2\widehat{a_1}) + \gamma(2\widehat{a_2}a_0 - 4a_2\widehat{a_0} + 2a_1\widehat{a_1}) + \widehat{a_1}a_0 + 4a_1\widehat{a_0}}{(\gamma^2 a_2 - 2\gamma a_1 + a_0)^3},$$


$$a_i = {x^k}^T A^i x^k, i = 0,1,2,3$$


In [84]:
def F2(x):
    return (x**3 * (-2.*a2*a2t) + x**2 * (-3. * a2 * a1t) + x * (2.*a2t * a0 - 4. * a2 * a0t + 2. * a1 * a1t) + a1t * a0 + 4. * a1 * a0t) / (a2 * x**2 - 2. * x * a1 + a0)**3

# Далее идет сам метод
Шаги вычисляются в соответствии с написанным в статье, с подстраховской на тот случай, если коэффициент при $\gamma^2$ обратится в ноль (помечено "!")

In [85]:
x = x0

In [86]:
print_rate = 20
for i in range(100):
    a0 = x.T.dot(x)
    a1 = x.T.dot(np.dot(A,x))
    a2 = x.T.dot(np.dot(A_sq,x))
    a3 = x.T.dot(np.dot(A_th,x))
    a2t = a2**2 - a3 * a1
    a1t = a3 * a0 - a2 * a1
    a0t = a1**2 - a0 * a2
    """
    print a0
    print a1
    print a2
    print a3
    print '_____'
    print a2t
    print a1t
    print a0t
    """
    if (a2**2 - a3 * a1) != 0:
        if i % print_rate == 0:
            print 'case quadr'
        gamma1 = ((a2 * a1 - a3 * a0) + np.sqrt((a2 * a1 - a3 * a0)**2 - (a2**2 - a3 * a1)*(a1**2 - a0 * a2)))/2. / (a2**2 - a3 * a1)
        gamma2 = ((a2 * a1 - a3 * a0) - np.sqrt((a2 * a1 - a3 * a0)**2 - (a2**2 - a3 * a1)*(a1**2 - a0 * a2)))/2. / (a2**2 - a3 * a1)
        gamma_list = []
        for gamma in [gamma1, gamma2]:
            if gamma >=0 and F2(gamma) > 0:
                gamma_list.append(gamma)
        #print 'gamma_list len = ', len(gamma_list)
        if len(gamma_list) > 0:
            final_gamma = gamma_list[np.random.randint(len(gamma_list))]
        else:
            final_gamma = 0
            if i % print_rate == 0:
                print 'there is no gamma to move'
        if i % print_rate == 0:
            print 'quadr gamma = ', final_gamma 
        x = (x - final_gamma * np.dot(A, x))/ np.linalg.norm(x - final_gamma * np.dot(A, x))
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    if  (a2**2 - a3 * a1) == 0 and (a3 * a0 - a2 * a1) != 0:
        if i % print_rate == 0:
            print 'case linear'
        gamma = (a1**2 - a0 * a2)/(a3 * a0 - a2 * a1)
        if gamma > 0 and F2(gamma) > 0:
                x = (x - gamma * np.dot(A, x))/ np.linalg.norm(x - gamma * np.dot(A, x))
        else:
            if i % print_rate == 0:
                print 'you got no suitable gamma'
    if (a3 * a0 - a2 * a1) == 0 and (a2**2 - a3 * a1) == 0:
        if i % print_rate == 0:
            print 'you got rekt'
            print 'iternum = ', i
            print '(a3 * a0 - a2 * a1)', (a3 * a0 - a2 * a1)
            print '(a2**2 - a3 * a1)', (a2**2 - a3 * a1)
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
    if i % print_rate == 0:
        print 'iter_num = ', i
        print x
print x

case quadr
quadr gamma =  0.00249996875078
iter_num =  0
[ 0.70710568 -0.00176774  0.70710568]
case quadr
quadr gamma =  0.0361240783536
iter_num =  20
[ 0.70750225 -0.00704957  0.70667593]
case quadr
quadr gamma =  0.00781413789622
iter_num =  40
[ 0.70846229 -0.00708255  0.70571313]
case quadr
quadr gamma =  0.00633383907512
iter_num =  60
[ 0.7087422  -0.00708611  0.70543198]
case quadr
quadr gamma =  0.0166937589469
iter_num =  80
[ 0.70881926 -0.0070881   0.70535453]
[ 0.70884536 -0.00708806  0.7053283 ]


# Далее вывод: оптимальное значение, оптимальная точка и проверка на принадлежность сфере

In [87]:
print 'f* = ', x.dot(np.dot(A,x))
print 'x* = ', x
print 'norm check: ', np.linalg.norm(x)

f* =  -0.00501224773857
x* =  [ 0.70884536 -0.00708806  0.7053283 ]
norm check:  1.0
